In [1]:
from embeddings import compute_umap, compute_tsne
import tifffile as tif
import numpy as np
import pandas as pd
import utils
import visualizer
import utils
from sklearn.cluster import KMeans


/Users/soitu/anaconda3/envs/cellpose3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-13 15:44:19.751196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# read images and convert to numpy array
data_path = '../data/'
images_path = data_path + 'images/'

images_list = utils.list_files(images_path)

images = []
for image in images_list:
    images.append(tif.imread(images_path + image))
images = np.array(images)
print('Shape of images:', images.shape)

# convert to nx(d1*d2) array
images = images.reshape(images.shape[0], -1)
print('Shape of images:', images.shape)

Shape of images: (1000, 28, 28)
Shape of images: (1000, 784)


In [ ]:
# Compute UMAP

umap_embeddings = compute_umap(images, n_components=10, n_neighbors=15, min_dist=0.1, metric='euclidean')

# Compute t-SNE
#tsne_embeddings = compute_tsne(images, n_components=10, perplexity=30, metric='euclidean')

# save all in a pandas dataframe
df = pd.DataFrame()
df['image'] = images_list
df['umap_embedding_1'] = umap_embeddings[:, 0]
df['umap_embedding_2'] = umap_embeddings[:, 1]



/Users/soitu/anaconda3/envs/cellpose3/lib/python3.8/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [20]:
# add columns for the visualizer: x pixels, y pixels, url of the image in base64, some clustering

df['url'] = np.nan
df['x_px'] = 128
df['y_px'] = 128

for i, row in df.iterrows():
    df.at[i, 'url'] = utils.encode_image_to_base64(images_path + row['image'])

# k-means clustering on the reduced_embeddings
kmeans = KMeans(n_clusters=10, random_state=0).fit(df[['umap_embedding_1', 'umap_embedding_2']])
df['k_means'] = kmeans.labels_
df.to_csv(data_path + 'embeddings.csv', index=False)


/Users/soitu/anaconda3/envs/cellpose3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [3]:
df = pd.read_csv(data_path + 'embeddings.csv')
visualizer.viz_umap_plot(data_df=df, embeddings_col_0='umap_embedding_1', embeddings_col_1='umap_embedding_2', name_col='image', x_px_col='x_px', y_px_col='y_px', url_col='url')

Dash app running on http://127.0.0.1:8050/
